# G₂ Quaternionic Sampling v5

## Intrinsic S³ Geometry via Quaternions

**Key Innovation**: Sample on S¹ × S³ × S³ using **quaternionic coordinates** with **geodesic distances**.

### Problem with v4
- v4 projected S³ (4D) → ℝ³ (3D), losing ~80% of volume
- Used Euclidean chord distance instead of geodesic arc distance
- Result: λ₁×H* ≈ 5-6.5 instead of target 14

### v5 Solution
- S³ ≅ SU(2) = unit quaternions {q = w + xi + yj + zk : |q| = 1}
- Geodesic distance: d(q₁,q₂) = 2·arccos(|⟨q₁,q₂⟩|)
- No projection needed - work in intrinsic 4D coordinates

### Target
- λ₁ × H* = 14 = dim(G₂)
- det(g) = 65/32 = 2.03125

---
*GIFT Framework - Geometric Information Field Theory*

In [ ]:
# Cell 1: Imports and GPU Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, Optional
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Check for GPU
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✓ GPU available via CuPy")
except ImportError:
    GPU_AVAILABLE = False
    print("○ Running on CPU")

print(f"NumPy version: {np.__version__}")

In [ ]:
# Cell 2: GIFT Constants
@dataclass
class GIFTConstants:
    """Topological constants from K₇ = TCS G₂-manifold."""
    # Betti numbers
    b2: int = 21          # H²(K₇) - from TCS Mayer-Vietoris
    b3: int = 77          # H³(K₇) - from TCS Mayer-Vietoris
    
    # Derived constants
    H_star: int = 99      # b₂ + b₃ + 1 (Hodge number)
    dim_G2: int = 14      # dim(G₂) holonomy group
    dim_K7: int = 7       # Manifold dimension
    
    # Metric determinant
    det_g: float = 65/32  # = 2.03125 (exact from G₂ structure)
    
    # Target eigenvalue
    target_lambda_H: int = 14  # λ₁ × H* = dim(G₂)
    
    # S³ eigenvalue (round metric, radius 1)
    lambda1_S3: float = 3.0  # First non-zero eigenvalue on S³

CONSTANTS = GIFTConstants()
print(f"H* = {CONSTANTS.H_star}")
print(f"Target: λ₁ × H* = {CONSTANTS.target_lambda_H}")
print(f"det(g) = {CONSTANTS.det_g} = 65/32")

In [ ]:
# Cell 3: Quaternion Algebra
class Quaternion:
    """
    Quaternion operations for S³ ≅ SU(2).
    
    A quaternion q = w + xi + yj + zk is represented as (w, x, y, z).
    Unit quaternions |q| = 1 form S³.
    """
    
    @staticmethod
    def normalize(q: np.ndarray) -> np.ndarray:
        """Normalize quaternion(s) to unit length."""
        if q.ndim == 1:
            return q / np.linalg.norm(q)
        return q / np.linalg.norm(q, axis=1, keepdims=True)
    
    @staticmethod
    def random_unit(n: int, seed: Optional[int] = None) -> np.ndarray:
        """
        Sample n uniformly distributed unit quaternions.
        
        Method: Gaussian in ℝ⁴, then normalize.
        This gives uniform distribution on S³.
        """
        if seed is not None:
            np.random.seed(seed)
        q = np.random.randn(n, 4)
        return Quaternion.normalize(q)
    
    @staticmethod
    def inner_product(q1: np.ndarray, q2: np.ndarray) -> np.ndarray:
        """Quaternion inner product ⟨q₁, q₂⟩ = Σ q1ᵢ q2ᵢ."""
        if q1.ndim == 1 and q2.ndim == 1:
            return np.dot(q1, q2)
        return np.sum(q1 * q2, axis=-1)
    
    @staticmethod
    def geodesic_distance(q1: np.ndarray, q2: np.ndarray) -> np.ndarray:
        """
        Geodesic distance on S³ between quaternions.
        
        d(q₁, q₂) = 2·arccos(|⟨q₁, q₂⟩|)
        
        Note: Factor of 2 because S³ has diameter π (not 2π).
        The |·| handles q and -q representing same rotation.
        """
        dot = np.abs(Quaternion.inner_product(q1, q2))
        # Clamp for numerical stability
        dot = np.clip(dot, 0.0, 1.0)
        return 2.0 * np.arccos(dot)
    
    @staticmethod
    def chord_distance(q1: np.ndarray, q2: np.ndarray) -> np.ndarray:
        """Euclidean chord distance in ℝ⁴ (for comparison)."""
        if q1.ndim == 1 and q2.ndim == 1:
            return np.linalg.norm(q1 - q2)
        return np.linalg.norm(q1 - q2, axis=-1)

# Test quaternion operations
q_test = Quaternion.random_unit(5, seed=42)
print(f"Sample quaternions shape: {q_test.shape}")
print(f"Norms (should be 1): {np.linalg.norm(q_test, axis=1)}")
print(f"Geodesic d(q₀,q₁) = {Quaternion.geodesic_distance(q_test[0], q_test[1]):.4f}")
print(f"Chord d(q₀,q₁) = {Quaternion.chord_distance(q_test[0], q_test[1]):.4f}")

In [ ]:
# Cell 4: TCS Quaternionic Sampler
class TCSQuaternionicSampler:
    """
    Sample on S¹ × S³ × S³ using intrinsic quaternionic coordinates.
    
    TCS structure: K₇ = M₁ ∪_neck M₂
    Neck region: S¹ × K3 ≈ locally S¹ × S³ × S³
    
    Coordinates:
    - θ ∈ [0, 2π): S¹ neck angle
    - q₁ ∈ S³: first quaternionic 3-sphere
    - q₂ ∈ S³: second quaternionic 3-sphere (scaled by ratio)
    """
    
    def __init__(self, n_points: int = 5000, seed: Optional[int] = None):
        self.n_points = n_points
        self.seed = seed
        
        # Storage
        self.theta = None   # S¹ angles
        self.q1 = None      # S³₁ quaternions
        self.q2 = None      # S³₂ quaternions
        
    def sample(self) -> None:
        """Generate sample points on S¹ × S³ × S³."""
        if self.seed is not None:
            np.random.seed(self.seed)
        
        # S¹: uniform angles
        self.theta = np.random.uniform(0, 2*np.pi, self.n_points)
        
        # S³₁: unit quaternions
        self.q1 = Quaternion.random_unit(self.n_points)
        
        # S³₂: unit quaternions (scaling applied in distance calculation)
        self.q2 = Quaternion.random_unit(self.n_points)
        
        print(f"Sampled {self.n_points} points on S¹ × S³ × S³")
        print(f"  θ shape: {self.theta.shape}")
        print(f"  q₁ shape: {self.q1.shape}")
        print(f"  q₂ shape: {self.q2.shape}")
    
    def compute_distance_matrix(self, ratio: float = 1.0, 
                                 alpha: float = 1.0,
                                 use_geodesic: bool = True) -> np.ndarray:
        """
        Compute pairwise distance matrix with TCS metric.
        
        Metric: ds² = α dθ² + β ds₁² + γ ds₂²
        where:
        - α: neck (S¹) scale
        - β = 1: S³₁ scale (normalized)
        - γ = ratio²: S³₂ scale
        
        Parameters:
        - ratio: size ratio S³₂/S³₁ (target √2 for GIFT)
        - alpha: neck scale parameter
        - use_geodesic: if True, use geodesic distance on S³; else chord
        """
        n = self.n_points
        D = np.zeros((n, n))
        
        # Precompute S¹ distances (circular)
        theta_diff = np.abs(self.theta[:, None] - self.theta[None, :])
        d_S1 = np.minimum(theta_diff, 2*np.pi - theta_diff)
        
        # Compute S³ distances
        if use_geodesic:
            # Geodesic distances (intrinsic)
            d_S3_1 = np.zeros((n, n))
            d_S3_2 = np.zeros((n, n))
            
            # Vectorized computation
            for i in range(n):
                d_S3_1[i, :] = Quaternion.geodesic_distance(self.q1[i], self.q1)
                d_S3_2[i, :] = Quaternion.geodesic_distance(self.q2[i], self.q2)
        else:
            # Chord distances (Euclidean in ℝ⁴) - for comparison
            d_S3_1 = squareform(pdist(self.q1, 'euclidean'))
            d_S3_2 = squareform(pdist(self.q2, 'euclidean'))
        
        # Combined TCS metric distance
        # ds² = α dθ² + 1·ds₁² + ratio²·ds₂²
        D = np.sqrt(alpha * d_S1**2 + d_S3_1**2 + (ratio**2) * d_S3_2**2)
        
        return D

# Test sampler
sampler = TCSQuaternionicSampler(n_points=100, seed=42)
sampler.sample()
D_test = sampler.compute_distance_matrix(ratio=1.0)
print(f"\nDistance matrix shape: {D_test.shape}")
print(f"Mean distance: {np.mean(D_test[D_test > 0]):.4f}")

In [ ]:
# Cell 5: Graph Laplacian with Geodesic Weights
class GeodesicGraphLaplacian:
    """
    Graph Laplacian using geodesic-based kernel weights.
    
    The key innovation: weights based on intrinsic S³ geometry,
    not ambient Euclidean distances.
    """
    
    def __init__(self, sigma: Optional[float] = None, k_neighbors: int = 20):
        """
        Parameters:
        - sigma: Gaussian kernel bandwidth (auto if None)
        - k_neighbors: for k-NN graph construction
        """
        self.sigma = sigma
        self.k_neighbors = k_neighbors
        
    def compute(self, D: np.ndarray, normalized: bool = True) -> sp.csr_matrix:
        """
        Compute graph Laplacian from distance matrix.
        
        Parameters:
        - D: pairwise distance matrix
        - normalized: if True, use symmetric normalized Laplacian
        
        Returns:
        - L: sparse Laplacian matrix
        """
        n = D.shape[0]
        
        # Auto sigma: median of k-nearest neighbor distances
        if self.sigma is None:
            knn_dists = np.sort(D, axis=1)[:, 1:self.k_neighbors+1]
            self.sigma = np.median(knn_dists)
        
        # Gaussian kernel weights
        W = np.exp(-D**2 / (2 * self.sigma**2))
        np.fill_diagonal(W, 0)  # No self-loops
        
        # k-NN sparsification for efficiency
        for i in range(n):
            # Keep only k largest weights
            idx = np.argsort(W[i])[:-(self.k_neighbors+1)]
            W[i, idx] = 0
        
        # Symmetrize
        W = (W + W.T) / 2
        
        # Degree matrix
        d = np.sum(W, axis=1)
        
        if normalized:
            # Symmetric normalized Laplacian: L = I - D^{-1/2} W D^{-1/2}
            d_inv_sqrt = np.where(d > 0, 1.0 / np.sqrt(d), 0)
            D_inv_sqrt = sp.diags(d_inv_sqrt)
            W_sparse = sp.csr_matrix(W)
            L = sp.eye(n) - D_inv_sqrt @ W_sparse @ D_inv_sqrt
        else:
            # Unnormalized Laplacian: L = D - W
            L = sp.diags(d) - sp.csr_matrix(W)
        
        return L.tocsr()
    
    def spectrum(self, L: sp.csr_matrix, k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """
        Compute k smallest eigenvalues and eigenvectors.
        
        Returns:
        - eigenvalues: sorted ascending
        - eigenvectors: corresponding eigenvectors
        """
        eigenvalues, eigenvectors = eigsh(L, k=k, which='SM')
        idx = np.argsort(eigenvalues)
        return eigenvalues[idx], eigenvectors[:, idx]

print("GeodesicGraphLaplacian class defined")

In [ ]:
# Cell 6: TCS Metric with det(g) = 65/32
class TCSMetricNormalizer:
    """
    Ensure TCS metric has det(g) = 65/32.
    
    For metric g = diag(α, β, β, β, γ, γ, γ) on S¹ × S³ × S³:
    det(g) = α × β³ × γ³
    
    Given ratio r = γ/β, we solve for α, β to get det(g) = 65/32.
    """
    
    def __init__(self, target_det: float = 65/32):
        self.target_det = target_det
    
    def compute_scales(self, ratio: float) -> Tuple[float, float, float]:
        """
        Compute metric scales (α, β, γ) for given ratio.
        
        We set β = 1 (normalization), γ = ratio, then solve for α.
        det(g) = α × 1³ × ratio³ = 65/32
        => α = (65/32) / ratio³
        """
        beta = 1.0
        gamma = ratio
        alpha = self.target_det / (beta**3 * gamma**3)
        
        return alpha, beta, gamma
    
    def verify_det(self, alpha: float, beta: float, gamma: float) -> float:
        """Verify determinant computation."""
        return alpha * (beta**3) * (gamma**3)

# Test metric normalizer
normalizer = TCSMetricNormalizer()
for r in [1.0, np.sqrt(2), 2.0]:
    a, b, g = normalizer.compute_scales(r)
    det = normalizer.verify_det(a, b, g)
    print(f"ratio={r:.4f}: α={a:.4f}, β={b:.4f}, γ={g:.4f}, det(g)={det:.6f}")

In [ ]:
# Cell 7: Main Experiment Runner
class QuaternionicExperiment:
    """
    Run spectral experiments with quaternionic TCS sampling.
    """
    
    def __init__(self, n_points: int = 5000, k_neighbors: int = 30, seed: int = 42):
        self.n_points = n_points
        self.k_neighbors = k_neighbors
        self.seed = seed
        
        self.sampler = TCSQuaternionicSampler(n_points, seed)
        self.normalizer = TCSMetricNormalizer()
        self.laplacian = GeodesicGraphLaplacian(k_neighbors=k_neighbors)
        
        self.results = []
        
    def run_single(self, ratio: float, use_geodesic: bool = True) -> dict:
        """
        Run experiment for a single ratio value.
        
        Parameters:
        - ratio: S³₂/S³₁ size ratio
        - use_geodesic: True for geodesic distances, False for chord
        """
        # Get metric scales ensuring det(g) = 65/32
        alpha, beta, gamma = self.normalizer.compute_scales(ratio)
        
        # Sample points (reuse same sample for fair comparison)
        if self.sampler.theta is None:
            self.sampler.sample()
        
        # Compute distance matrix with TCS metric
        # Note: We scale the S¹ component by sqrt(alpha) and S³₂ by ratio
        # The distance formula: ds² = α dθ² + β ds₁² + γ ds₂²
        # With β=1, γ=ratio, we have ds² = α dθ² + ds₁² + ratio² ds₂²
        D = self.sampler.compute_distance_matrix(
            ratio=ratio,
            alpha=alpha,
            use_geodesic=use_geodesic
        )
        
        # Reset sigma for each experiment
        self.laplacian.sigma = None
        
        # Compute Laplacian and spectrum
        L = self.laplacian.compute(D, normalized=True)
        eigenvalues, _ = self.laplacian.spectrum(L, k=10)
        
        # First non-zero eigenvalue
        lambda1 = eigenvalues[1]  # eigenvalues[0] ≈ 0
        lambda1_H = lambda1 * CONSTANTS.H_star
        
        result = {
            'ratio': ratio,
            'alpha': alpha,
            'beta': beta,
            'gamma': gamma,
            'det_g': self.normalizer.verify_det(alpha, beta, gamma),
            'lambda1': lambda1,
            'lambda1_H': lambda1_H,
            'use_geodesic': use_geodesic,
            'sigma': self.laplacian.sigma,
            'eigenvalues': eigenvalues[:5].tolist()
        }
        
        return result
    
    def run_sweep(self, ratios: np.ndarray, use_geodesic: bool = True) -> list:
        """
        Sweep over multiple ratio values.
        """
        results = []
        
        # Sample once
        self.sampler.sample()
        
        for i, ratio in enumerate(ratios):
            print(f"  [{i+1}/{len(ratios)}] ratio = {ratio:.4f}...", end=" ")
            result = self.run_single(ratio, use_geodesic)
            results.append(result)
            print(f"λ₁×H* = {result['lambda1_H']:.4f}")
        
        self.results = results
        return results

print("QuaternionicExperiment class defined")

In [ ]:
# Cell 8: Run Main Experiment - Geodesic vs Chord Comparison
print("="*60)
print("EXPERIMENT: Quaternionic TCS Sampling v5")
print("="*60)
print(f"Target: λ₁ × H* = {CONSTANTS.target_lambda_H}")
print(f"H* = {CONSTANTS.H_star}")
print("="*60)

# Parameters
N_POINTS = 5000
K_NEIGHBORS = 30
SEED = 42

# Ratio sweep: 1.0 to √2 in 11 steps
ratios = np.linspace(1.0, np.sqrt(2), 11)

# Create experiment
experiment = QuaternionicExperiment(N_POINTS, K_NEIGHBORS, SEED)

print(f"\n[1/2] Running with GEODESIC distances (intrinsic S³)...")
results_geodesic = experiment.run_sweep(ratios, use_geodesic=True)

print(f"\n[2/2] Running with CHORD distances (Euclidean ℝ⁴)...")
# Reset sampler for fair comparison with same points
experiment.sampler = TCSQuaternionicSampler(N_POINTS, SEED)
results_chord = experiment.run_sweep(ratios, use_geodesic=False)

print("\n" + "="*60)
print("DONE")
print("="*60)

In [ ]:
# Cell 9: Results Analysis
print("\n" + "="*60)
print("RESULTS COMPARISON")
print("="*60)

print("\n{:<10} {:>15} {:>15} {:>10}".format(
    "ratio", "λ₁×H* (geod)", "λ₁×H* (chord)", "det(g)"))
print("-"*55)

for rg, rc in zip(results_geodesic, results_chord):
    print("{:<10.4f} {:>15.4f} {:>15.4f} {:>10.5f}".format(
        rg['ratio'], rg['lambda1_H'], rc['lambda1_H'], rg['det_g']))

print("-"*55)
print(f"Target: λ₁×H* = {CONSTANTS.target_lambda_H}")

# Find closest to target
geod_vals = [r['lambda1_H'] for r in results_geodesic]
chord_vals = [r['lambda1_H'] for r in results_chord]

best_geod_idx = np.argmin(np.abs(np.array(geod_vals) - CONSTANTS.target_lambda_H))
best_chord_idx = np.argmin(np.abs(np.array(chord_vals) - CONSTANTS.target_lambda_H))

print(f"\nClosest to target (geodesic): ratio={ratios[best_geod_idx]:.4f}, λ₁×H*={geod_vals[best_geod_idx]:.4f}")
print(f"Closest to target (chord): ratio={ratios[best_chord_idx]:.4f}, λ₁×H*={chord_vals[best_chord_idx]:.4f}")

In [ ]:
# Cell 10: Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: λ₁×H* vs ratio
ax1 = axes[0]
ax1.plot(ratios, geod_vals, 'b-o', label='Geodesic (intrinsic)', linewidth=2, markersize=8)
ax1.plot(ratios, chord_vals, 'r--s', label='Chord (Euclidean)', linewidth=2, markersize=8)
ax1.axhline(y=CONSTANTS.target_lambda_H, color='g', linestyle=':', linewidth=2, label=f'Target = {CONSTANTS.target_lambda_H}')
ax1.axhline(y=2*np.pi**2, color='orange', linestyle='-.', linewidth=1.5, label=f'2π² ≈ {2*np.pi**2:.2f}')
ax1.axvline(x=np.sqrt(2), color='purple', linestyle=':', alpha=0.5, label=f'√2 ≈ {np.sqrt(2):.4f}')

ax1.set_xlabel('S³ Size Ratio (γ/β)', fontsize=12)
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Quaternionic TCS Sampling: Spectral Gap', fontsize=14)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0.95, 1.45])

# Plot 2: det(g) verification
ax2 = axes[1]
det_vals = [r['det_g'] for r in results_geodesic]
ax2.plot(ratios, det_vals, 'k-o', linewidth=2, markersize=8)
ax2.axhline(y=65/32, color='g', linestyle='--', linewidth=2, label=f'Target = 65/32 = {65/32:.5f}')

ax2.set_xlabel('S³ Size Ratio (γ/β)', fontsize=12)
ax2.set_ylabel('det(g)', fontsize=12)
ax2.set_title('Metric Determinant Verification', fontsize=14)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)
ax2.set_ylim([2.02, 2.04])

plt.tight_layout()
plt.savefig('g2_quaternionic_v5_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved: g2_quaternionic_v5_results.png")

In [ ]:
# Cell 11: Save Results
output = {
    'timestamp': datetime.now().isoformat(),
    'notebook': 'G2_Quaternionic_Sampling_v5',
    'method': 'Quaternionic S³ with geodesic distances',
    'parameters': {
        'n_points': N_POINTS,
        'k_neighbors': K_NEIGHBORS,
        'seed': SEED
    },
    'constants': {
        'H_star': CONSTANTS.H_star,
        'dim_G2': CONSTANTS.dim_G2,
        'det_g_target': CONSTANTS.det_g,
        'target_lambda_H': CONSTANTS.target_lambda_H
    },
    'results_geodesic': {
        'ratios': ratios.tolist(),
        'lambda1_H': [r['lambda1_H'] for r in results_geodesic],
        'det_g': [r['det_g'] for r in results_geodesic],
        'alpha': [r['alpha'] for r in results_geodesic]
    },
    'results_chord': {
        'ratios': ratios.tolist(),
        'lambda1_H': [r['lambda1_H'] for r in results_chord],
        'det_g': [r['det_g'] for r in results_chord]
    },
    'analysis': {
        'best_geodesic_ratio': ratios[best_geod_idx],
        'best_geodesic_lambda_H': geod_vals[best_geod_idx],
        'best_chord_ratio': ratios[best_chord_idx],
        'best_chord_lambda_H': chord_vals[best_chord_idx]
    }
}

# Save to file
output_path = 'outputs/g2_quaternionic_v5_results.json'
import os
os.makedirs('outputs', exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(output, f, indent=2)

print(f"Results saved to: {output_path}")
print("\n" + "="*60)
print("EXPERIMENT COMPLETE")
print("="*60)

In [ ]:
# Cell 12: High-Resolution Sweep (Optional)
RUN_HIGH_RES = False  # Set to True for detailed analysis

if RUN_HIGH_RES:
    print("\n" + "="*60)
    print("HIGH-RESOLUTION SWEEP")
    print("="*60)
    
    # Fine sweep around √2
    ratios_hires = np.linspace(1.0, 2.0, 51)
    
    experiment_hires = QuaternionicExperiment(
        n_points=10000,  # More points
        k_neighbors=50,   # More neighbors
        seed=42
    )
    
    print("\nRunning high-res geodesic sweep...")
    results_hires = experiment_hires.run_sweep(ratios_hires, use_geodesic=True)
    
    # Save high-res results
    hires_output = {
        'timestamp': datetime.now().isoformat(),
        'notebook': 'G2_Quaternionic_Sampling_v5_HighRes',
        'parameters': {
            'n_points': 10000,
            'k_neighbors': 50,
            'n_ratios': 51
        },
        'results': {
            'ratios': ratios_hires.tolist(),
            'lambda1_H': [r['lambda1_H'] for r in results_hires],
            'det_g': [r['det_g'] for r in results_hires]
        }
    }
    
    with open('outputs/g2_quaternionic_v5_hires.json', 'w') as f:
        json.dump(hires_output, f, indent=2)
    
    print("\nHigh-res results saved.")
else:
    print("\nSkipping high-resolution sweep.")
    print("Set RUN_HIGH_RES = True to enable.")

## Summary

### Key Innovations in v5

1. **Quaternionic Sampling**: S³ ≅ SU(2) = unit quaternions
   - No projection needed (full 4D coordinates preserved)
   - Uniform distribution on S³ via normalized Gaussians

2. **Geodesic Distances**: d(q₁,q₂) = 2·arccos(|⟨q₁,q₂⟩|)
   - Intrinsic arc length on S³
   - Compared with Euclidean chord distance

3. **TCS Metric Normalization**: det(g) = 65/32 enforced
   - Metric: g = diag(α, 1, 1, 1, r², r², r²)
   - α = (65/32) / r³ ensures correct determinant

### Expected Results

| Method | Expected λ₁×H* | Notes |
|--------|---------------|-------|
| v4 (projection) | 5-6.5 | Lost dimension info |
| v5 geodesic | 10-16? | Intrinsic geometry |
| v5 chord | 8-12? | Still ℝ⁴ embedding |
| Target | 14 | dim(G₂) |

---
*GIFT Framework - Quaternionic TCS Analysis*